In [1]:
import sys
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import mlfoundry as mlf

In [2]:
mlf.login(api_key="OGQ1NDI4ZGMtZGY3My00ZWEyLTg2NGMtZjA3OTQzNDkzZDRiOmZjNjc4Mw==", relogin=True)
client = mlf.get_client()

Writing API key at /home/ec2-user/.mlfoundry/credentials.netrc


In [3]:
class MovieLensDataset(torch.utils.data.Dataset):
    def __init__(self, df, mean_rating=3.58):
        self.df = df
        self.mean_rating = mean_rating
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x = torch.tensor([row.user_id, row.movie_id], dtype=torch.int32)
        y = torch.tensor(row.rating - self.mean_rating, dtype=torch.float)
        return x, y

In [4]:
train = pd.read_csv("data/ml1m_train.csv")
train_dataset = MovieLensDataset(train)
test = pd.read_csv("data/ml1m_test.csv")
test_dataset = MovieLensDataset(test)

In [5]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items,1)
        torch.nn.init.xavier_uniform_(self.user_factors.weight)
        torch.nn.init.xavier_uniform_(self.item_factors.weight)
        self.user_biases.weight.data.fill_(0.)
        self.item_biases.weight.data.fill_(0.)
        
    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        return pred.squeeze()

In [6]:
run = client.get_run("cloud/recommendation-system/matrix-factorization-factor5-45")
mf_model = run.get_model()

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/176/85f1694846d3432490623e444b76066f/


In [19]:
class DeepFactorization(torch.nn.Module):
    
    def __init__(self, mf: MatrixFactorization, e_factors, layer_size, n_users, n_items):
        super().__init__()
        self.mf = mf
        self.mf.user_factors.weight.requires_grad = False
        self.mf.user_biases.weight.requires_grad = False
        self.mf.item_factors.weight.requires_grad = False
        self.mf.item_biases.weight.requires_grad = False
        
        self.user_extra_factors = torch.nn.Embedding(n_users, e_factors)
        self.item_extra_factors = torch.nn.Embedding(n_items, e_factors)
        
        self.linear1 = torch.nn.Linear(12 + 2 * e_factors, layer_size)
        self.dropout = torch.nn.Dropout(p=0.2)
        self.linear2 = torch.nn.Linear(layer_size, 1)
        
        torch.nn.init.xavier_uniform_(self.user_extra_factors.weight)
        torch.nn.init.xavier_uniform_(self.item_extra_factors.weight)
        self.user_extra_factors.weight.data.fill_(0.)
        self.item_extra_factors.weight.data.fill_(0.)
        
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        self.linear1.bias.data.fill_(0.01)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        self.linear2.bias.data.fill_(0.01)
        
    def forward(self, user, item):
        joint_embed = torch.cat((self.mf.user_biases(user),
                                 self.mf.user_factors(user),
                                 self.mf.item_biases(item),
                                 self.mf.item_factors(item),
                                 self.user_extra_factors(user),
                                 self.item_extra_factors(item)), dim=1)
        output = self.linear1(joint_embed)
        output = self.dropout(output)
        output = self.linear2(output)
        return output.squeeze()

In [20]:
reviewers = 6041
books = 3953
batch_size = 128

In [21]:
device = torch.device("cpu")

In [22]:
def train_loop(model, train_batch, label_batch, loss_func, optimizer):
    prediction = model(train_batch[:,0].to(device), train_batch[:,1].to(device))
    loss = loss_func(prediction, label_batch.to(device))    
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

In [25]:
def train(e_factors, layer_size, learning_rate=0.02):
    run = client.create_run(project_name="recommendation-system", run_name=f"deep-factorization-2-layer", log_system_metrics=False)
    run.log_params({"learning_rate": learning_rate})
    run.set_tags({"type": "deep"})
    
    model = DeepFactorization(mf_model, e_factors, layer_size, reviewers, books)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    model.to(device)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    epoches = 100
    last_test_loss = 10000
    for epoch in range(0, epoches):
        # Training
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        train_count = 1
        train_loss = 0.
        for i, (train_batch, label_batch) in pbar:
            loss = train_loop(model, train_batch, label_batch, loss_func, optimizer)
            train_loss += loss
            train_count += 1
            pbar.set_description(f'Train loss at {epoch} batch {i}: {train_loss/train_count}')

        # Calculate test loss
        pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
        test_loss = 0.
        test_count = 1
        for i,( test_batch, label_batch) in pbar:
            with torch.no_grad():
                prediction = model(test_batch[:,0].to(device), test_batch[:,1].to(device))
                loss = loss_func(prediction, label_batch.to(device))
                test_loss += loss.item()
                test_count += 1
                pbar.set_description(f'Test loss at {epoch} batch {i}: {test_loss/test_count}')
        
        run.log_metrics(
            metric_dict={
                "train_loss": train_loss/train_count,
                "test_loss": test_loss/test_count,
            }, step=epoch
        )
        # Early stopping
        current_test_loss = test_loss/test_count
        if last_test_loss - current_test_loss < 0.001:
            print(f"Stopping Training. Last")
            break
        last_test_loss = current_test_loss
    run.log_model(model=model, framework="pytorch")
    run.end()

In [26]:
train(e_factors=3, layer_size=10, learning_rate=0.001)

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/176/09c234477f9f4be6995becb8ef4587f1/
[mlfoundry] 2022-07-07T09:34:20+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-07-07T09:34:20+0000 INFO Run 'cloud/recommendation-system/deep-factorization-2-layer-82' has started.
[mlfoundry] 2022-07-07T09:34:20+0000 INFO Parameters logged successfully
[mlfoundry] 2022-07-07T09:34:20+0000 INFO Tags set successfully


Test loss at 0 batch 1562: 0.8297074876554177: 100%|██████████| 1563/1563 [00:23<00:00, 65.36it/s]


[mlfoundry] 2022-07-07T09:36:25+0000 INFO Metrics logged successfully


Test loss at 1 batch 1562: 0.8289621773430759: 100%|██████████| 1563/1563 [00:23<00:00, 65.79it/s]


[mlfoundry] 2022-07-07T09:38:30+0000 INFO Metrics logged successfully
Stopping Training. Last
[mlfoundry] 2022-07-07T09:38:33+0000 INFO Model logged successfully
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/176/09c234477f9f4be6995becb8ef4587f1/
